In [1]:
# import warnings
# warnings.filterwarnings("ignore")


import math
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec

import ipywidgets as widgets

%matplotlib inline

import panel as pn

#Qt5Agg

# Suggested by Hoxbro at Panel
plt.close('all')
plt.switch_backend("agg")
# plt.switch_backend("Qt5Agg")


pn.extension()

import lasio

from bokeh.models.formatters import PrintfTickFormatter
from bokeh.models.widgets import Button

from scipy.optimize import minimize
from scipy.interpolate import interp1d

In [2]:
log=lasio.read(r'D:\Programming Sabda\Project Interactive\Project Interactive\Python Poseidon\Crossplot sensitive of Overpressure\New Data\Well\Poseidon North 1_logs.las')
log=log.df().reset_index()
log.head()

,DEPTH,CALI,CHKS,RHOB,GR,UNKNOWN:1,UNKNOWN:2,NPHI,DT:1,DT:2,RES,ILD,SDT
0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67049.3906,NaN,NaN,NaN
1,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3047.6997,NaN,NaN,NaN
2,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1523.8000,NaN,NaN,NaN
3,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1523.7500,NaN,NaN,NaN
4,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1523.7494,NaN,NaN,NaN


In [3]:
import pandas as pd
MW = pd.read_csv(r'D:\Programming Sabda\Project Interactive\Project Interactive\Python Poseidon\Pressure Data\excel\excel\POSEIDON NORTH1.csv', delimiter=';')
print(MW)

     No.       date   MDRT (m)  TVDRT (m)  Pore Pressure (ppg)  \
0    1.0  20/03/2014    568.34     568.34                  8.4   
1    2.0  21/03/2014   1321.00    1321.00                  8.5   
2    3.0  22/03/2014   1427.00    1426.90                  8.5   
3    4.0  23/03/2014   1918.00    1917.80                  8.5   
4    5.0  24/03/2014   2190.00    2189.70                  8.5   
..   ...         ...       ...        ...                  ...   
994  NaN         NaN       NaN        NaN                  NaN   
995  NaN         NaN       NaN        NaN                  NaN   
996  NaN         NaN       NaN        NaN                  NaN   
997  NaN         NaN       NaN        NaN                  NaN   
998  NaN         NaN       NaN        NaN                  NaN   

     Mud Weight (ppg)  LOT (ppg)   MDRT (m).1  TVDRT (m).1  Pore (psi/ft)  \
0                 8.7        NaN  1864.632606  1864.632606         0.4368   
1                 8.6        NaN  4333.989640  4333.9

In [4]:
top=pd.read_csv('D:/Programming Sabda/Project Interactive/Project Interactive/Python Poseidon/Pressure Data/2.Top Formation/Poseidon North TOP.csv', delimiter=';')
Depthshifttop=abs(top['TVDSS'])-486+60
toptvdss=np.zeros(len(top['MDRT\n']))
nametop=[top['Formation Name']]
print(nametop)
labs = ['Piertszoon / Sea Bed'
                ,'Tasman Marl Member top'
                 , 'Oliver Formation top'
     ,'Van Der Wall Limestone Member top'
                 ,'Read Limestone Member'
          ,'Dampier Limestone Member top'
        ,'Allouarn Limestone Member top'
              ,'Heywood Limestone Member'
                ,'Baudin Marl Member top'
                 ,'Johnson Formation top'
                     ,'Johnson Sand Mbr'
                   ,'WGFP Formation top'
               ,'Jamieson Formation top'
          ,'Echuca Shoals Formation top'
               ,'Brewster Shale Mbr top'
                     , 'Montara Mbr top'
                       ,'Golden Mbr top'
                  ,'Triassic (Nome) top'
               ,'Triassic (Challis) top'
                                   ,'TD']

[0                  Piertszoon I Sea Bed
1                Tasman Marl Member top
2                  Oliver Formation top
3     Van Der Wall Limestone Member top
4                 Read Limestone Member
5          Dampier Limestone Member top
6         Allouarn Limestone Member top
7              Heywood Limestone Member
8                Baudin Marl Member top
9                 Johnson Formation top
10                     Johnson Sand Mbr
11                   WGFP Formation top
12               Jamieson Formation top
13          Echuca Shoals Formation top
14               Brewster Shale Mbr top
15                      Montara Mbr top
16                       Golden Mbr top
17                  Triassic (Nome) top
18               Triassic (Challis) top
19                                   TD
Name: Formation Name, dtype: object]


In [5]:
MDT = pd.read_csv('D:/Programming Sabda/Project Interactive/Project Interactive/Python Poseidon/Pressure Data/excel//5. Poseidon North MDT&XPT/Poseidon North MDT.csv', delimiter=';')
print(MDT)
depthshiftmdt=MDT['Pretest DEPTH TVDMSL']-486

   Pretest DEPTH MD  Pretest DEPTH TVDMSL  Final Pressure (Psia)
0            4921.9                4898.4                 9851.3
1            4923.6                4900.1                 9860.4
2            4924.8                4901.3                 9864.4
3            4924.8                4901.3                 9864.4
4            4932.1                4908.6                 9924.8
5            4933.8                4910.3                 9932.9
6            4933.8                4910.3                 9834.5
7            4934.3                4910.8                 9930.8
8            4935.0                4911.5                 9950.4


In [6]:
from scipy.optimize import curve_fit
def traugott(z, a, b):
    r"""
    estimate density with depth

    Parameters
    ----------
    depth : 1-d ndarray
    a, b: scalar

    Notes
    -----
    .. math:: \overline{\rho (h)}=16.3+{h/3125}^{0.6}

    gives the average sediment density in pounds per gallon (ppg) mud weight
    equivalent between the sea floor and depth h (in feet) below the sea floor.

    So, density variation with depth takes the form [2]_:

    .. math:: \rho(z) = {\rho}_{0} + a{z}^{b}

    .. [2] Traugott, Martin. "Pore/fracture pressure determinations in deep
       water." World Oil 218.8 (1997): 68-70.
    """
    # rho0 = 2.65
    return 2.2 + a * z**b

In [7]:
def traugott_trend(depth, a, b, kb=0, wd=0):
    depth_shift = np.array(depth) - kb - wd
    density = traugott(depth_shift, a, b)
    mask = depth_shift < 0
    density[mask] = np.nan
    return density

In [8]:
def optimize_traugott( fit_start, fit_stop, kb=0, wd=0):
    """
    Fit density variation against depth with Traugott equation

    Parameters
    ----------
    den_log : Log
        Density log
    fit_start, fit_stop : float
        start and end depth for fitting
    kb : float
        kelly bushing height in meters
    wd : float
        water depth in meters

    Returns
    -------
    a, b : float
        Traugott equation coefficients
    """
    logfit = log[(log.DEPTH >= fit_start) & (log.DEPTH <= fit_stop)]
    
    depth = np.array(logfit.DEPTH )
    den = np.array(logfit.RHOB_300 )

    mask = np.isfinite(den)
    den_finite = den[mask]
    depth_finite = depth[mask]

    depth_finite_shift = depth_finite - kb - wd

    popt, _ = curve_fit(traugott, depth_finite_shift, den_finite)
    a, b = popt

    return a, b

In [9]:
kb=-22
wd=486
depthshift = log[(log.DEPTH >= wd)] - kb - wd
log['DEPTHSHIFT']= depthshift.DEPTH

In [10]:
d_seawater= 1.024
g=9.81
Mudline = 486 # Seafloor: This an assumed value showing first recorded data points in the well logs.
# Pressure at the Seafloor
P_mudline = d_seawater * g * Mudline*1000
P_mudline

4882083.84

In [11]:
def med_filt(window=4000):
    min_per = 1 # to avoid getting NaN if there is a single NaN, I request to have at least min_per real numbers in the window
    log[f'RHOB_{window}']= log['RHOB'].rolling(window,min_periods=min_per, center=True).median()
med_filt(300)
med_filt(100)

In [12]:
a,b=optimize_traugott( 1000, 3231, kb=kb, wd=wd)

In [13]:
Dens_trend = traugott_trend(
    np.array(log['DEPTH']), a,b, 
    kb=kb, wd=wd)

C:\Users\sabda\AppData\Local\Temp\ipykernel_49436\3628000629.py:26: RuntimeWarning: invalid value encountered in power
  return 2.2 + a * z**b


In [14]:
new_data = np.full_like(Dens_trend, np.nan)
new_data[0:958] = Dens_trend[0:958]
old_data = np.array(log['RHOB'])
new_data[958:4931] = old_data[958:4931]
new_data[4931:] = Dens_trend[4931:]

# new_data[27265:] =Dens_trend[27265:]
log['NewRHOB'] = new_data

In [15]:
fig,ax=plt.subplots(figsize=(5,7))
# ax.plot(Dens_trend,log['DEPTH'])
ax.scatter(log.NewRHOB,log.DEPTH)
# ax.scatter(log['RHOB'],log['DEPTH'], c='b',label='RHOB')
ax.set_xlabel('Density',fontsize='large')
ax.set_ylabel('Depth [ft]')
# ax.set_ylim(0,4400)
ax.invert_yaxis()
ax.set_xlabel('g/cc')
plt.show()

C:\Users\sabda\AppData\Local\Temp\ipykernel_49436\1953227869.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [16]:
fig,ax=plt.subplots(figsize=(5,7))
ax.plot(Dens_trend,log['DEPTH'])
ax.scatter(log['RHOB'],log['DEPTH'], c='b',label='RHOB')
ax.set_xlabel('Density',fontsize='large')
ax.set_ylabel('Depth [ft]')
# ax.set_ylim(0,4400)
ax.invert_yaxis()
ax.set_xlabel('g/cc')

Text(0.5, 0, 'g/cc')

In [17]:
loglito=log.dropna(subset=['NewRHOB'])

In [18]:
def overburden_pressure(TVDs, bulk_density, p0 = 0):
    """"
    depth : depth in m
    density : bulk density in g/cc
    p0 : pressure at 0. If 0 is mudline -> p0 is mudline
    """
    g = 9.81
    dz = np.concatenate([np.array([np.diff(TVDs).mean()]),np.diff(TVDs)])
    return p0 + np.cumsum(bulk_density*1000*g*dz)

In [19]:

log['Hydrostatic_Pressure_at_bit2']= d_seawater*g*1000*log['DEPTH']
log['Lithostatic_Pressure_at_bit']=overburden_pressure(TVDs=loglito['DEPTHSHIFT'], 
                                    bulk_density=loglito['NewRHOB'],
                                    p0 = P_mudline)
log['Overburden_Gradient'] = log['NewRHOB'] *g*d_seawater*1000 
log['Hydrostatic_Pressure_Gradient']=d_seawater*g*1000  # in Pa / m

In [20]:
def interactive_pressure_plot(gr_clean, gr_shale,DTm, DTml, c,Eaton_coefficient):
    # =============================================================================
    # We will create several displays (GR,sonic with NCT trends, pore pressure with mudweight vs DEPTH )
    # in order to see and choose the right parameters
    # for our clay/clean formations as well 
    # =============================================================================
    fig = plt.figure(figsize=(30, 25))
    gs = gridspec.GridSpec(1, 4)
    ax1 = fig.add_subplot(gs[:,0])
    ax2 = fig.add_subplot(gs[:,1])
    ax3 = fig.add_subplot(gs[:,2])
    ax4 = fig.add_subplot(gs[:,3],sharey = ax1)
    curve_names = ['GR (API)', 'DT (us/ft)', 'Pressure (ppg)','Vshale']
    # =============================================================================
    # GR Display
    # =============================================================================
    ax1.plot(log['GR'],log['DEPTH'], color='green', linewidth=1)
    ax1.set_ylim(min(log['DEPTH']), max(log['DEPTH']))
    ax1.set_xlim(0, 150)
    ax1.set_xlabel('GR (API)', fontsize=30)
    ax1.set_ylabel('DEPTH (m)', fontsize=30)
    ax1.tick_params(axis='both', which='major', labelsize=20)
    ax1.invert_yaxis()
    ax1.grid()
    ax1.axvline(x=gr_clean, color='black', linestyle='--', linewidth=1)
    ax1.axvline(x=gr_shale, color='red', linestyle='--', linewidth=1)
    
    # =============================================================================
    # SONIC & NCT SONIC Display
    # =============================================================================
    ax2.plot(log['DT:1'],log['DEPTH'], color='blue', linewidth=1)
    # Calculate VshGR for each depth
    ax2.plot(DTc_NCT(DTm, DTml,log.DEPTH,c),log['DEPTH'], color='blue', linewidth=1)
    # Only plot DT_30 where VshGR > 0.5
    mask = vshgr(log.GR, gr_clean, gr_shale) > 0
    if 'DT_100' in log.columns:
        ax2.scatter(log.loc[mask, 'DT_100'], log.loc[mask, 'DEPTH'], c='r', label='DTC shale (filt 30)')
    # ax2.plot(dt_NCT,log['DEPTH'], color='red', linewidth=3)
    ax2.set_ylim(min(log['DEPTH']), max(log['DEPTH']))
    ax2.set_xlim(30, 250)
    ax2.set_xlabel('DT (us/ft)', fontsize=30)
    ax2.tick_params(axis='both', which='major', labelsize=20)
    ax2.invert_yaxis()
    ax2.grid()
    # =============================================================================
    # Pore Pressure & Mud Weight Display
    # =============================================================================
    
    # new=put_ppg_above_hydrostatic(compute_PPG_from_Eaton_Transit (log['Overburden_Gradient'],log['Hydrostatic_Pressure_Gradient'],DTc_NCT(DTm,DTml,j,c),log.loc[mask,'DT_100'], Eaton_coefficient), d_water = d_seawater)
    new=put_ppg_above_hydrostatic(compute_PPG_from_Eaton_Transit (log['Overburden_Gradient'],log['Hydrostatic_Pressure_Gradient'],DTc_NCT(DTm,DTml,log.DEPTH,c),log['DT_100'], Eaton_coefficient), d_water = d_seawater)
    

    log['PPFinal']=new*log.DEPTH*(145.038/1e6)
    # log['PPGFinal']=log['PPFinal']/0.052/(log['DEPTH']*3.2808399)
    ax3.plot(log['PPFinal'],log['DEPTH'],linestyle='-', c='r',label='Hydrostatic Pressure')
    ax3.plot(log['Hydrostatic_Pressure_at_bit2']*(145.038/1e6),log['DEPTH'],linestyle='-', c='b',label='Hydrostatic Pressure')
    ax3.plot(log['Lithostatic_Pressure_at_bit']*(145.038/1e6),log['DEPTH'],linestyle='-', c='g',label='Overburden Pressure')
    ax3.plot(MW['Mud Weight(psi) MD'], MW['TVDRT (m)']-25,marker='^', color='gray', linewidth=1)
    ax3.set_ylim(min(log['DEPTH']), max(log['DEPTH']))
    ax3.set_xlabel('Pressure (ppg)', fontsize=30)
    ax3.tick_params(axis='both', which='major', labelsize=20)
    ax3.invert_yaxis()
    ax3.grid()
    #Set up the common elements between the subplots
    ax4.plot(vshgr(log.GR, gr_clean, gr_shale), log.DEPTH, label ='Vsh_GR',color="green", linewidth = 1.0)
    ax4.set_title('Calculated Shale Volumes', color = 'blue')
    ax4.legend(loc='best',fontsize='x-small')
    ax4.set_xlim(0,1)
    ax4.invert_yaxis()
    ax4.grid(True)
    ax4.set_xlabel('Vsh [v.v]')

    for i, ax in enumerate(fig.axes):
        ax.set_ylim(0, 5300) # Set the depth range
        ax.scatter(toptvdss,abs(top['TVDSS']),marker="+",s=100000,c="gold")
        ax.xaxis.set_ticks_position("top")
        ax.xaxis.set_label_position("top")
        ax.set_xlabel(curve_names[i],fontsize=5)
        ax.invert_yaxis()
        # if i == 2:
        #     ax.spines["top"].set_position(("axes", 1.08))
        # else:
        #     ax.grid()
            
    for i, txt in enumerate(labs):
        ax1.annotate(txt, (toptvdss[i],abs(top['TVDSS'])[i]), fontsize=6, color='black')

    
    # Adjust figure size to be proportional to the number of tracks and depth range, but limit max size
    # Further reduce font sizes and figure size for compact display
    for ax in fig.axes:
        ax.title.set_fontsize(7)
        ax.xaxis.label.set_fontsize(7)
        ax.yaxis.label.set_fontsize(7)
        ax.tick_params(axis='both', which='major', labelsize=7)
        legend = ax.get_legend()
        if legend:
            for text in legend.get_texts():
                text.set_fontsize(6)
    # Make figure even smaller and more proportional
    n_tracks = 4
    depth_range = max(log['DEPTH']) - min(log['DEPTH'])
    width_per_track = 9  # much smaller width per track
    height_per_1000m = 3  # much smaller height per 1000m
    max_width = 10
    max_height = 10
    fig.set_size_inches(
        min(width_per_track * n_tracks, max_width),
        min(height_per_1000m * (depth_range / 1000), max_height)
    )
    # plt.show()
    # plt.tight_layout()
    return fig

In [21]:
"""
===============================================================================
 === 3. Calculate Volume of Shale

 The first step of a basic well log analysis will begin with the volume
 of Shale calculation.

 There are several methods used to perform that, clasified as: - Single
 methods: Gamma Ray Log, Spontaneous Potential, vshrt or by - Dual
 Methods: Neutron-Density, it is based on a N-D Crossplot, where a clean
 line and a clay point will be defined, as you will see in the graphs
 below.
===============================================================================
"""
# Define GR clean and shale values (adjust as needed for your well)
# gr_clean = 70   # Example value for clean sand
# gr_shale = 110  # Example value for shale
gr_clean, gr_shale      = 45  , 120     
# Define NCT Sonic
DTm,DTml,c = 25.60228734,180.61230556,2.95000e-04
Eaton_coefficient = 1.2  # Typical value between 1.0 and 1.5
#vshgr
def vshgr(gr_log, gr_clean, gr_shale):
    vshgr=(gr_log - gr_clean)/(gr_shale - gr_clean)      #Linear Gamma Ray
    return vshgr
#filtering DT
def med_filt(window=4000):
    min_per = 1 # to avoid getting NaN if there is a single NaN, I request to have at least min_per real numbers in the window
    log[f'DT_{window}']= log['DT:1'].rolling(window,min_periods=min_per, center=True).median()
# def vshdt():
#     mask = log['vshgr'] > 0
#     if 'DT_30' in log.columns:
#         vshdt=[log.loc[mask, 'DT_30'] & log.loc[mask, 'DEPTH']]
def DTc_NCT(DTm, DTml,TVDs, c): 
    
    """
    DESCRIBE IN A FEW WORDS WHAT YOU WANT TO DO
    Estimate the normal compaction DT for any depth
    input is depth
    DTml, DTm and c are parameters
    
    DTm = Transit time in Shale matrix us/m
    Intercept = DTml = Transit time in Mudline (z=0) us/m
    1/c = caracterstic compaction depth 
    z: depth mbsf
    """  
    return DTm + (DTml - DTm) *  np.exp(-c*TVDs) 

def compute_PPG_from_Eaton_Transit (obg,hbg,DTc_NCT,DT, Eaton_coefficient ):
    """
    OBG = Overburden gradient = Lithostatic_Pressure_Gradient
    HBG = Hydrostatic_Pressure_Gradient
    DTc = shale resistivity
    DT = shale resistivity Normal trend line 
    """
    return obg-(obg-hbg)*(DTc_NCT/DT)**Eaton_coefficient
def put_ppg_above_hydrostatic(ppg, d_water = d_seawater):
    """    
    If PPG is less than hydrostatic, it is put above hydrostatic
    """
    return np.where(ppg<d_water*1000*g,d_water*1000*g,ppg)

def put_ppg_to_hydrostatic_on_top(ppg,zlim=0,z=log['DEPTHSHIFT'], d_water = d_seawater):
    """    
    If depth is less than a threshold depth, ppg is set to hydrostatic
    """
    return np.where(z<0,d_water*1000*g,ppg)

# def compute_pore_pressure_from_TVD(TVDs,DT_column,
#                                    hydro_until=None,Eaton_coefficient):
#     ppg = compute_ppg_from_TVD(TVDs,DT_column=DT_column,Eaton_coefficient=Eaton_coefficient)
#     if hydro_until is not None:
#         ppg = put_ppg_to_hydrostatic_on_top(ppg,zlim=hydro_until)
#     p = P_mudline + ppg * TVDs
#     return ppg
top_hydro = 0

# new=put_ppg_above_hydrostatic(ppg_100_DT_cut, d_water = d_seawater)
# log['PPFinal']=new*log.DEPTH*(145.038/1e6)
# log['PPGFinal']=log['PPFinal']/0.052/(log['DEPTH']*3.2808399)
#initialize various Vsh
#vshgr_temp,vshnd_temp, vshrt_temp, vshsp_temp , vshnmphi_temp =[],[],[],[], []
vshgr_temp =[]
NCT_temp=[]
PP_temp=[]
# ===== this is an example of for a,b in zip(alist,blist): ====================
# alist = ['a1', 'a2', 'a3']
# blist = ['b1', 'b2', 'b3']
#
# for a, b in zip(alist, blist):
#     print a, b
#
# =============================================================================
# =============================================================================
# This is key for the input of log data to be used in Vsh calculations
# =============================================================================
#for (i,j,k,l,m,n) in zip(logs.GR,logs.NPHI,logs.RHOB,logs.ILD,logs.SP,logs.MPHI):
#    vshgr_temp.append(vshgr(i, gr_clean, gr_shale))
#    vshnd_temp.append(vshnd(j,k,neut_matrix,den_matrix,neut_fl,den_fl,neut_shale,den_shale))
#    vshsp_temp.append(vshsp(m, sp_clean, sp_shale))
#    vshnmphi_temp.append(vshnmphi(j, n, neut_shale, 0))
#
# =============================================================================
# This is key for the input of log data to be used in Vsh calculations
# =============================================================================
#                           i       j         k         l      m        n
med_filt(100)
for (i,j,k,l,m) in zip(log.GR,log.DEPTH,log['DT_100'],log['Overburden_Gradient'],log['Hydrostatic_Pressure_Gradient']):
    vshgr_temp.append(vshgr(i, gr_clean, gr_shale))
    NCT_temp.append(DTc_NCT(DTm,DTml,j,c))
    PP_temp.append(compute_PPG_from_Eaton_Transit (l,m,DTc_NCT(DTm,DTml,j,c),k, Eaton_coefficient ))
# =============================================================================


# =============================================================================
# This is the input of log data used in Vsh calculations
# =============================================================================
#======== test this as it prints GR, NPHI, RHOB, ILD, SP  =====================
#    print(i,j,k,l,m) #where i=GR, j=NPHI, k=RHOB, l=ILD and m=SP

#==============================================================================
# Add the calculated Vsh to the log DataFrame
log.is_copy = False # without will throw an exception
log['vshgr']=vshgr_temp
log['DTc_NCT']=NCT_temp
log['PP_Eaton_100']=PP_temp
del vshgr_temp,NCT_temp,PP_temp      #remove the arrays to free-up memory

log.head()

,DEPTH,CALI,CHKS,RHOB,GR,UNKNOWN:1,UNKNOWN:2,NPHI,DT:1,DT:2,...,RHOB_100,NewRHOB,Hydrostatic_Pressure_at_bit2,Lithostatic_Pressure_at_bit,Overburden_Gradient,Hydrostatic_Pressure_Gradient,DT_100,vshgr,DTc_NCT,PP_Eaton_100
0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67049.3906,...,NaN,NaN,220999.68,NaN,NaN,10045.44,NaN,NaN,179.609548,NaN
1,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3047.6997,...,NaN,NaN,231045.12,NaN,NaN,10045.44,NaN,NaN,179.564123,NaN
2,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1523.8000,...,NaN,NaN,241090.56,NaN,NaN,10045.44,NaN,NaN,179.518711,NaN
3,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1523.7500,...,NaN,NaN,251136.00,NaN,NaN,10045.44,NaN,NaN,179.473312,NaN
4,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1523.7494,...,NaN,NaN,261181.44,NaN,NaN,10045.44,NaN,NaN,179.427927,NaN


In [22]:
'''
Panel version that works
'''

grclean_slider  = pn.widgets.FloatSlider(start=0,end=150,step=0.5,value=gr_clean,name='gr_clean')
grshale_slider  = pn.widgets.FloatSlider(start=0,end=150,step=0.5,value=gr_shale,name='gr_shale')
DTm_slider  = pn.widgets.FloatSlider(start=0,end=100,step=1,value=DTm,name='DTm')
DTml_slider  = pn.widgets.FloatSlider(start=0,end=300,step=1,value=DTml,name='DTm1')
c_slider  = pn.widgets.FloatSlider(start=0.95000e-04,end=3.95000e-04,step=0.01000e-04,value=c,name='c')
Eaton_coefficient_slider  = pn.widgets.FloatSlider(start=0,end=5,step=0.1,value=Eaton_coefficient,name='Eaton_coefficient')
# To make the DTm, DTml, and c sliders update the plot, you need to update your interactive_pressure_plot function
# to accept DTm, DTml, and c as arguments and use them in the calculations.
# Update the function signature in cell 5 to:
# def interactive_pressure_plot(gr_clean, gr_shale, DTm, DTml, c):

# Then, update the pn.interact call to:
pn.interact(interactive_pressure_plot, gr_clean=grclean_slider, gr_shale=grshale_slider, DTm=DTm_slider, DTml=DTml_slider, c=c_slider, Eaton_coefficient=Eaton_coefficient_slider)



BokehModel(combine_events=True, render_bundle={'docs_json': {'b53959c5-356c-489b-9d73-2ac05d5f91e0': {'version…